In [1]:
import tensorflow as tf

C:\Users\nabu-pc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### VERY SIMPLE SESSION:

In [2]:
x = tf.Variable(3, name ="x")
y = tf.Variable(4, name ="y")
f= x*x*y + y +2   #here we don't calculate ANYTHING!!!

sess = tf.Session()  #we create a session
sess.run(x.initializer) #we inizialize the variables
sess.run(y.initializer)
result = sess.run(f) # HERE WE CALCULATE F() !!!
print(result)
sess.close() #Here we close the session!!

42


#### USING WITH BLOCK:

In [3]:
# We can use with block. Two benefits : 1- the session is automatically closed when with finishes 
                                       #2- the session is set as default session
with tf.Session() as sess:
    x.initializer.run() #it's equivalent to tf.get_default_session().run(x.initializer)
    y.initializer.run() 
    result = f.eval() # it's equivalent to calling tf.get_default_session().run(f)

#### GLOBAL VARIABLES:

In [4]:
# With global_variable_initializer() you don't have to inizialize every single variable

init = tf.global_variables_initializer()  #We prepare an init node. Here we don't initialize the variables!!!

with tf.Session() as sess:
    init.run() #actually initialize all the variables
    result = f.eval()

#### INTERACTIVE SESSION:

In [5]:
# Interactive session is different from a normal session: it is sets itself as the default session. So you don't need
# a with block

sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()  #we need to close the session!!!

42


#### PERFORMING A LINEAR REGRESSION: 

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]  #adding an extra feature with all 1 to all training instances

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")   #We create two Tensorflow constant nodes: data and targets
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)  # A vector (1-D) of Theta values 
                                                                       # Linear Algebra form (vector form): (XT*X)^-1*XT*y
                                                                       # Here we DON'T CALCULATE theta vector
                                                                       
with tf.Session() as sess:
    theta_value = theta.eval()   # Here we calculate Theta vector!!!
    
print(theta)
print(".........")
print(theta_value)

C:\Users\nabu-pc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensor("MatMul_2:0", shape=(9, 1), dtype=float32)
.........
[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


#### FUNCTION FOR CALCULATING GRADIENT MINI-BATCH:

In [2]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) #shape (batch_size,1) ,returns a vector of random integer from a continous distribution in a half open interval [)
    X_batch = scaled_housing_data_plus_bias[indices] # range of above random values are in the range [0,m)
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

#### CALCULATING THE GRADIENT MANUALLY WITH SAVING/RESTORING  (AND USING TENSORBOARD !!!)

In [3]:
from datetime import datetime
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

now = datetime.utcnow().strftime("%Y%m%d%H%M%S") #now = time in a specific format (YYYYMMDDHHMMSS)
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now ) # The path where we save the data: tf_logs/run-YYYYMMDDHHMMSS

batch_size = 100
n_batches = int(np.ceil(m / batch_size))  #int(arrotonda per eccesso(m/batch_size))

scaler = StandardScaler()
scaler.fit(housing_data_plus_bias)
scaled_housing_data_plus_bias  =scaler.transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape = (None, n + 1), name = "X") # We inserts a placeholder for a tensor that will be ALWAYS fed (alimentato) 
y = tf.placeholder(tf.float32, shape = (None, 1), name = "y")

theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name = "theta") # A tensor (variable) with shape (9,1) that takes random values from a uniform distribution .Values in the range (-1,1). 
y_pred = tf.matmul(X,theta, name ="predictions") #dot product -> X*theta. Shape -> (m,9)*(9,1) -> (m,1) 

with tf.name_scope("loss") as scope:    
    error = y_pred - y # Shape : (m,1)
    mse = tf.reduce_mean(tf.square(error), name ="mse")

gradients = 2/m*tf.matmul(tf.transpose(X), error)  # dot product XT*error -> (n,m)*(m,1) -> (n,1) -> a vector with Gradient values
training_op = tf.assign(theta, theta - learning_rate*gradients) #assign function permits to modify a variable (theta)

init = tf.global_variables_initializer()
saver = tf.train.Saver()  #Here we create the save node!!!

mse_summary = tf.summary.scalar('MSE', mse) # Creates a node in the graph that will evaluate the MSE value and writes it to a TensoBoard-compatible binary log string called Summary
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) # We create a filewriter that you use to write summaries to logfiles in the log directory
                                                                    # logdir = the patph of the logdir diretory. Second parameter = the graph we want to visualize
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "/tmp/my_model.ckpt") #We create a checkpoint each 100 epoch
        
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index %10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op,feed_dict={X: X_batch, y: y_batch}) #x_batch: shape (batch_size,n-1), y_batch: shape (batch_size,1) 
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt" )
    file_writer.close()

# For running Tensorboard, in the Anaconda Prompt IN /Progetto Forex/Python (our working directory) -> tensorboard --logdir tf_logs
    
# to restore the saved variable:
# with tf.Session() as sess:
#     saver.restore(sess, "/tmp/my_model_final.ckpt")

